In [1]:
from qbraid import device_wrapper, get_devices
from qbraid.devices._utils import SUPPORTED_VENDORS
from qbraid.devices.aws import BraketDeviceWrapper, BraketQuantumTaskWrapper
from qbraid.devices.google import CirqSimulatorWrapper, CirqResultWrapper
from qbraid.devices.ibm import QiskitBackendWrapper, QiskitJobWrapper

In [2]:
import cirq
import numpy as np
from braket.circuits import Circuit as BraketCircuit
from qiskit import QuantumCircuit as QiskitCircuit

In [15]:
import boto3
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
import networkx as nx
import dwave_networkx as dnx
from dwave.system.composites import EmbeddingComposite

In [8]:
my_bucket = "amazon-braket-592242689881"
my_prefix = "simulator-output"
s3_folder = (my_bucket, my_prefix)

In [27]:
sampler = BraketSampler(s3_folder, 'arn:aws:braket:::device/qpu/d-wave/Advantage_system1')
# sampler = BraketDWaveSampler(s3_folder, 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

In [19]:
embedded_sampler = EmbeddingComposite(sampler)

In [20]:
n = 5
m = 10
graph = nx.gnm_random_graph(n, m, seed=42)

In [21]:
# run the problem on D-Wave using BraketSampler
result = dnx.min_vertex_cover(graph, embedded_sampler, resultFormat="HISTOGRAM")
# or alternatively using BraketDWaveSampler
# result = dnx.min_vertex_cover(graph, embedded_sampler, answer_mode="histogram")
print('Result to MVC problem:', result)
print('Size of the vertex cover:', len(result))

Result to MVC problem: [0, 2, 3, 4]
Size of the vertex cover: 4


In [24]:
from networkx.classes.graph import Graph as nxGraph

In [26]:
isinstance(graph, nx.classes.graph.Graph)

True

In [ ]:
def device_wrapper_input(vendor):
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        for device in SUPPORTED_VENDORS[vendor][provider]:
            if provider != "D-Wave":
                data = (device, provider, vendor)
                input_list.append(data)
    return input_list

In [ ]:
def braket_circuit():
    circuit = BraketCircuit()
    circuit.h(0)
    circuit.ry(0, np.pi / 2)
    return circuit

In [ ]:
def cirq_circuit(meas=True):
    q0 = cirq.GridQubit(0, 0)
    def test_circuit():
        yield cirq.H(q0)
        yield cirq.Ry(rads=np.pi / 2)(q0)
        if meas:
            yield cirq.measure(q0, key='q0')
    circuit = cirq.Circuit()
    circuit.append(test_circuit())
    return circuit

In [ ]:
def qiskit_circuit(meas=True):
    circuit = QiskitCircuit(1, 1) if meas else QiskitCircuit(1)
    circuit.h(0)
    circuit.ry(np.pi / 2, 0)
    if meas:
        circuit.measure(0, 0)
    return circuit

In [ ]:
qiskit_circuit(meas=False).draw()

In [ ]:
braket_circuit = braket_circuit()
cirq_circuit = cirq_circuit()
qiskit_circuit = qiskit_circuit()

In [ ]:
get_devices(vendor="AWS")

In [ ]:
qbraid_device = device_wrapper("local_simulator_default", "Google", vendor="Google")

In [ ]:
qbraid_result = qbraid_device.run(cirq_circuit, repetitions=10)